<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Analysis using Vantage and OpenAI
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Analyzing consumer complaints using audio files conversations involves leveraging advanced technologies like large language models (LLM) and OpenAI to extract insights from unstructured data. This process can be applied to various sectors, including financial services, telecommunications, and e-commerce, to improve customer experience and protect consumer rights.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environment</li>
    <li>Configuring OpenAI Model</li>
    <li>Speech Analysis</li>
</ol>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Configuring the environment</b>

In [ ]:
!pip install -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd

# Progress bar
from tqdm import tqdm

# User input
import getpass

# Teradata utilities
from teradataml import *

# OpenAI API
import openai
from openai import OpenAI

# Display settings
display.max_rows = 5
pd.set_option('display.max_colwidth', None)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Configuring OpenAI Model</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell will prompt us for the following information:</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li><b>OpenAI API Key</b>: Enter your OpenAI API Key</li>
<ul>

In [ ]:
# Prompt user for OpenAI API key securely
openai.api_key = getpass.getpass(prompt="\nPlease enter your OpenAI API key: ")

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>2.1 Initialize the OpenAI Model</b>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li>The code below initializes a OpenAI client.</li>
</ul>

In [ ]:
client = OpenAI(
    api_key = openai.api_key
)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Speech Analysis</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell defines a prompt that instructs the LLM on steps to be performed.</p>

In [ ]:
prompt = '''
    review: “{}”
    Analyze the given review to extract relevant information and categorize it accordingly. Follow these steps, providing reasoning for each task:

    Complaint Categorization: Determine whether the review is a complaint or a non-complaint. A complaint is typically a statement expressing dissatisfaction or grievance, while a non-complaint does not contain such elements. Explain why you categorized it this way.

    Topic Classification: If the review is a complaint, identify the most appropriate topic from the following list:
        Mortgage Application
        Payment Trouble
        Mortgage Closing
        Report Inaccuracy
        Payment Struggle
        Select only one topic based on the central issue or subject of the complaint. Explain the reasoning behind your choice of topic.

    Sentiment Analysis: Determine the sentiment of the review, which can be:
        Positive
        Negative
        Neutral
        Sentiment refers to the emotional tone or attitude conveyed in the review. Describe the reasoning behind the sentiment you identified.

    Summarization: Summarize the review in one sentence. A good summary should capture the essence of the review, highlighting the main point or issue. Explain the key elements that formed the basis of your summary.

    Return your results in the following format. Only keep the headings in bold:

    #### Complaint:
    - [Provide the raw complaint from the input.]

    #### Classification:
    - [Specify whether it's a Complaint or Non-Complaint]
      - **Reasoning with Chain-of-Thought**: [Explain the reasoning behind the classification.]

    #### Topic:
    - [Specify the relevant topic]
      - **Reasoning with Chain-of-Thought**: [Provide additional context or reasoning related to the topic.]

    #### Sentiment:
    - [Specify the sentiment (e.g., Positive, Negative, Neutral)]
      - **Reasoning with Chain-of-Thought**: [Explain the rationale for the sentiment.]

    #### Summary:
    - [Summarize the complaint in a concise sentence]
      - **Reasoning with Chain-of-Thought**: [Include any relevant reasoning or context for the summary.]
'''

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell will transcribe the audio files using OpenAI <b>whisper-1</b> LLM model and call then call OpenAI <b>gpt-4o-mini</b> LLM to perform Complaint Analysis on it.</p>

In [ ]:
from IPython.display import display, Markdown, Audio, HTML

for file_name in sorted(os.listdir('./audio_files/')):
    audio_file= open(f"./audio_files/{file_name}", "rb")

    transcription = client.audio.transcriptions.create(
      model = "whisper-1",
      file = audio_file
    )

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an assistant that analyzes a given review as per the instructions."},
            {"role": "user", "content": prompt.format(transcription.text)}
        ]
    )
    output = completion.choices[0].message.content

    # Description HTML content
    desc = f'''<hr style="height:1px;border:none;background-color:#00233C;">
               <p style='font-size:18px;font-family:Arial;color:#00233C'><b>For file: {file_name}</b></p>'''

    # Create the HTML, audio, and markdown objects
    desc_display = HTML(desc)
    audio_display = Audio(f"./audio_files/{file_name}")
    markdown_display = Markdown(output)

    # Display all elements in a single output
    display(desc_display, audio_display, markdown_display)

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>